In [ ]:
!pip install requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 66.0 MB/s eta 0:00:00


#Soil Moisture Index

In [3]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from io import StringIO

# Function to fetch soil moisture data
def fetch_soil_moisture_data(latitude, longitude, date):
    url = f"https://api.meteomatics.com/{date}T00:00:00ZP1D/soil_moisture_index_-5cm:idx/{latitude},{longitude}/html?source=mix"
    username = "elnia_putri_stevia"
    password = "WQb0680Rcf"
    response = requests.get(url, auth=(username, password))

    if response.status_code == 200:
        return response.text
    else:
        print(f"Error: Unable to fetch soil moisture data for {date}. Status Code: {response.status_code}")
        return None

# Function to calculate average soil moisture
def calculate_average_soil_moisture(data):
    soup = BeautifulSoup(data, 'html.parser')
    csv_data = soup.find('pre', id='csv').text.strip()
    df = pd.read_csv(StringIO(csv_data), delimiter=';')
    return df['soil_moisture_index_-5cm:idx'].mean()

# Function to calculate percentage change
def calculate_percentage_change(yesterday, today):
    if yesterday == 0:
        return None  # Avoid division by zero
    percentage_change = ((today - yesterday) / yesterday) * 100
    return percentage_change

# List of provinces with their latitude and longitude
provinces = [
    {"name": "Aceh", "latitude": 4.695135, "longitude": 96.7493993},
    {"name": "Bali", "latitude": -8.4095178, "longitude": 115.188916},
    {"name": "Banten", "latitude": -6.4058172, "longitude": 106.0640179},
    {"name": "Bengkulu", "latitude": -3.5778471, "longitude": 102.3463875},
    {"name": "D.I. Yogyakarta", "latitude": -7.8753849, "longitude": 110.4262088},
    {"name": "D.K.I. Jakarta", "latitude": -6.211544, "longitude": 106.845172},
    {"name": "Gorontalo", "latitude": 0.6999372, "longitude": 122.4467238},
    {"name": "Jambi", "latitude": -1.4851831, "longitude": 102.4380581},
    {"name": "Jawa Barat", "latitude": -7.090911, "longitude": 107.668887},
    {"name": "Jawa Tengah", "latitude": -7.150975, "longitude": 110.1402594},
    {"name": "Jawa Timur", "latitude": -7.5360639, "longitude": 112.2384017},
    {"name": "Kalimantan Barat", "latitude": -0.2787808, "longitude": 111.4752851},
    {"name": "Kalimantan Selatan", "latitude": -3.0926415, "longitude": 115.2837585},
    {"name": "Kalimantan Tengah", "latitude": -1.6814878, "longitude": 113.3823545},
    {"name": "Kalimantan Timur", "latitude": 0.5387, "longitude": 116.419389},
    {"name": "Kalimantan Utara", "latitude": 3.0731, "longitude": 116.0414},
    {"name": "Kepulauan Bangka Belitung", "latitude": -2.7410513, "longitude": 106.4405872},
    {"name": "Kepulauan Riau", "latitude": 3.9456514, "longitude": 108.1428669},
    {"name": "Lampung", "latitude": -4.5585849, "longitude": 105.4068079},
    {"name": "Maluku", "latitude": -3.2384616, "longitude": 130.1452734},
    {"name": "Maluku Utara", "latitude": 1.5709993, "longitude": 127.8087693},
    {"name": "Nusa Tenggara Barat", "latitude": -8.6529334, "longitude": 117.3616476},
    {"name": "Nusa Tenggara Timur", "latitude": -8.6573819, "longitude": 121.0793705},
    {"name": "Papua", "latitude": -4.269928, "longitude": 138.0803529},
    {"name": "Papua Barat", "latitude": -1.3361154, "longitude": 133.1747162},
    {"name": "Riau", "latitude": 0.2933469, "longitude": 101.7068294},
    {"name": "Sulawesi Barat", "latitude": -2.8441371, "longitude": 119.2320784},
    {"name": "Sulawesi Selatan", "latitude": -3.6687994, "longitude": 119.9740534},
    {"name": "Sulawesi Tengah", "latitude": -1.4300254, "longitude": 121.4456179},
    {"name": "Sulawesi Tenggara", "latitude": -4.14491, "longitude": 122.174605},
    {"name": "Sulawesi Utara", "latitude": 0.6246932, "longitude": 123.9750018},
    {"name": "Sumatera Barat", "latitude": -0.7399397, "longitude": 100.8000051},
    {"name": "Sumatera Selatan", "latitude": -3.3194374, "longitude": 103.914399},
    {"name": "Sumatera Utara", "latitude": 2.1153547, "longitude": 99.5450974}
]

# Get today's and yesterday's date
today_date = datetime.now().strftime('%Y-%m-%d')
yesterday_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')

# Loop through each province and fetch soil moisture data
for province in provinces:
    print(f"Fetching data for {province['name']}...")

    # Fetch data for yesterday and today
    yesterday_data = fetch_soil_moisture_data(province['latitude'], province['longitude'], yesterday_date)
    today_data = fetch_soil_moisture_data(province['latitude'], province['longitude'], today_date)

    if yesterday_data and today_data:
        avg_soil_moisture_yesterday = calculate_average_soil_moisture(yesterday_data)
        avg_soil_moisture_today = calculate_average_soil_moisture(today_data)

        # Calculate percentage change
        percentage_change = calculate_percentage_change(avg_soil_moisture_yesterday, avg_soil_moisture_today)

        # Print results
        print(f"{province['name']} - Soil Moisture (Yesterday): {avg_soil_moisture_yesterday:.2f}")
        print(f"{province['name']} - Soil Moisture (Today): {avg_soil_moisture_today:.2f}")

        if percentage_change is not None:
            if percentage_change > 0:
                print(f"Soil moisture increased by {percentage_change:.2f}%")
            else:
                print(f"Soil moisture decreased by {abs(percentage_change):.2f}%")
        else:
            print("No percentage change due to zero moisture yesterday.")
    else:
        print(f"Could not fetch data for {province['name']}.")


Fetching data for Aceh...
Aceh - Soil Moisture (Yesterday): 1.18
Aceh - Soil Moisture (Today): 1.14
Soil moisture decreased by 3.31%
Fetching data for Bali...
Bali - Soil Moisture (Yesterday): 0.43
Bali - Soil Moisture (Today): 0.45
Soil moisture increased by 4.38%
Fetching data for Banten...
Banten - Soil Moisture (Yesterday): 0.20
Banten - Soil Moisture (Today): 0.13
Soil moisture decreased by 36.11%
Fetching data for Bengkulu...
Bengkulu - Soil Moisture (Yesterday): 0.94
Bengkulu - Soil Moisture (Today): 1.06
Soil moisture increased by 11.74%
Fetching data for D.I. Yogyakarta...
D.I. Yogyakarta - Soil Moisture (Yesterday): 0.36
D.I. Yogyakarta - Soil Moisture (Today): 0.34
Soil moisture decreased by 7.80%
Fetching data for D.K.I. Jakarta...
D.K.I. Jakarta - Soil Moisture (Yesterday): 0.58
D.K.I. Jakarta - Soil Moisture (Today): 0.65
Soil moisture increased by 11.04%
Fetching data for Gorontalo...
Gorontalo - Soil Moisture (Yesterday): 0.94
Gorontalo - Soil Moisture (Today): 0.88
Soi

In [4]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from io import StringIO

# Function to fetch soil moisture data
def fetch_soil_moisture_data(latitude, longitude, date):
    url = f"https://api.meteomatics.com/{date}T00:00:00ZP1D/soil_moisture_index_-5cm:idx/{latitude},{longitude}/html?source=mix"
    username = "elnia_putri_stevia"
    password = "WQb0680Rcf"
    response = requests.get(url, auth=(username, password))

    if response.status_code == 200:
        return response.text
    else:
        print(f"Error: Unable to fetch soil moisture data for {date}. Status Code: {response.status_code}")
        return None

# Function to calculate average soil moisture
def calculate_average_soil_moisture(data):
    soup = BeautifulSoup(data, 'html.parser')
    csv_data = soup.find('pre', id='csv').text.strip()
    df = pd.read_csv(StringIO(csv_data), delimiter=';')
    return df['soil_moisture_index_-5cm:idx'].mean()

# Function to calculate percentage change
def calculate_percentage_change(yesterday, today):
    if yesterday == 0:
        return None  # Avoid division by zero
    percentage_change = ((today - yesterday) / yesterday) * 100
    return percentage_change

# List of provinces with their latitude and longitude
provinces = [
    {"name": "Sumatera Barat", "latitude": -0.7399397, "longitude": 100.8000051},
    {"name": "Sumatera Selatan", "latitude": -3.3194374, "longitude": 103.914399},
    {"name": "Sumatera Utara", "latitude": 2.1153547, "longitude": 99.5450974}
]

# Get today's and yesterday's date
today_date = datetime.now().strftime('%Y-%m-%d')
yesterday_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')

# Loop through each province and fetch soil moisture data
for province in provinces:
    print(f"Fetching data for {province['name']}...")

    # Fetch data for yesterday and today
    yesterday_data = fetch_soil_moisture_data(province['latitude'], province['longitude'], yesterday_date)
    today_data = fetch_soil_moisture_data(province['latitude'], province['longitude'], today_date)

    if yesterday_data and today_data:
        avg_soil_moisture_yesterday = calculate_average_soil_moisture(yesterday_data)
        avg_soil_moisture_today = calculate_average_soil_moisture(today_data)

        # Calculate percentage change
        percentage_change = calculate_percentage_change(avg_soil_moisture_yesterday, avg_soil_moisture_today)

        # Print results
        print(f"{province['name']} - Soil Moisture (Yesterday): {avg_soil_moisture_yesterday:.2f}")
        print(f"{province['name']} - Soil Moisture (Today): {avg_soil_moisture_today:.2f}")

        if percentage_change is not None:
            if percentage_change > 0:
                print(f"Soil moisture increased by {percentage_change:.2f}%")
            else:
                print(f"Soil moisture decreased by {abs(percentage_change):.2f}%")
        else:
            print("No percentage change due to zero moisture yesterday.")
    else:
        print(f"Could not fetch data for {province['name']}.")


Fetching data for Sumatera Barat...
Sumatera Barat - Soil Moisture (Yesterday): 1.22
Sumatera Barat - Soil Moisture (Today): 1.29
Soil moisture increased by 5.57%
Fetching data for Sumatera Selatan...
Sumatera Selatan - Soil Moisture (Yesterday): 0.46
Sumatera Selatan - Soil Moisture (Today): 0.86
Soil moisture increased by 86.65%
Fetching data for Sumatera Utara...
Sumatera Utara - Soil Moisture (Yesterday): 0.96
Sumatera Utara - Soil Moisture (Today): 0.93
Soil moisture decreased by 2.79%
